# Discretization Example

## Imports and definitions

In [0]:
import tensorflow as tf
import numpy as np

from attacks import adv_lspga
from discretization_utils import discretize_uniform

## Model definition

In [0]:
def model(x, is_training=True):
      with tf.variable_scope('SmallModel', reuse=tf.AUTO_REUSE):
        # conv layer 1
        conv1 = tf.layers.conv2d(
            inputs=x,
            filters=16,
            kernel_size=[8, 8],
            strides=(2, 2),
            padding='same',
            activation=tf.nn.relu,
            name='conv1')

        flat = tf.contrib.layers.flatten(conv1)
        dense = tf.layers.dense(inputs=flat, units=32, name='dense')
        dropout = tf.layers.dropout(inputs=dense, rate=.2, training=is_training)
        logits = tf.layers.dense(inputs=dropout, units=10, name='logits')
        return logits

## Adversarial training with discrete attack

In [0]:
# Get batch of data from data pipeline
features, labels = get_data()

# features is of shape [batch_size, height, width, channels]
# labels is of shape [batch_size, classes]

levels = 8
features_disc = discretize_uniform(features, levels=levels, thermometer=True) 

# features_disc is of shape [batch_size, height, width, channels * levels]

# Set projection_fn to identity
projection_fn = tf.identity

# Set phase of the model, i.e. whether training or testing
phase = tf.placeholder(tf.bool, name='phase')

# Set attack parameters
steps = 10
eps = 0.3
attack_step = 0.1

# Get attacked images
features_disc_adv = adv_lspga(features_disc, model, discretize_uniform, 
                              projection_fn, levels, phase, steps, eps, 
                              attack_step)

# Get model prediction on adversarial features
logits_adv = model(features_adv, is_training=phase)

# Define the loss for adversarial training
adv_loss = tf.losses.softmax_cross_entropy(
    onehot_labels=labels,
    logits=logits_adv)

# Define the train op
tf.train.AdamOptimizer(
    learning_rate=1e-3).minimize(adv_loss)